This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_001_Zr_code93_sto__Li_110_slab_heavy
- Initial lattice height: 74.46 Å
- Number of atoms: 900

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 15 Å
- Li top: frozen top 15 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 14-07-2025


In [9]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read("llzo_li_balanced_sliced/LLZO_001_Zr_code93_sto__Li_110_slab_heavy.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness = 15.0  # in Å
llzo_z_threshold = z_min + freeze_thickness
li_z_threshold = z_max - freeze_thickness

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)



Freezing 648 atoms out of 900
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_10383/74757100.py:26: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [5]:
prediction = chgnet.predict_structure(structure)

for key, unit in [
    ("energy", "eV/atom"),
    ("forces", "eV/A"),
    ("stress", "GPa"),
    ("magmom", "mu_B"),
]:
    print(f"CHGNet-predicted {key} ({unit}):\n{prediction[key[0]]}\n")

CHGNet-predicted energy (eV/atom):
-2.4268932342529297

CHGNet-predicted forces (eV/A):
[[ 0.0334  0.0568 -0.0308]
 [ 0.0605 -0.0243 -0.0104]
 [ 0.0327  0.0785 -0.2108]
 ...
 [-0.3813 -0.2442 -0.2677]
 [-0.3591 -0.1395 -1.1941]
 [-0.0857  0.1973 -0.5799]]

CHGNet-predicted stress (GPa):
[[-27.7557  -0.0051   0.0033]
 [ -0.0051  -8.9737  -1.5942]
 [  0.0033  -1.5942  -9.3248]]

CHGNet-predicted magmom (mu_B):
[0.0037 0.009  0.0102 0.0181 0.0119 0.0065 0.0003 0.002  0.0009 0.0059
 0.0126 0.0036 0.0031 0.0011 0.002  0.0018 0.012  0.0094 0.0085 0.0072
 0.0098 0.0095 0.0053 0.0069 0.0076 0.0084 0.009  0.0096 0.009  0.0093
 0.009  0.0091 0.007  0.0082 0.0094 0.0093 0.0094 0.009  0.0085 0.0088
 0.006  0.0062 0.0053 0.0052 0.0087 0.0085 0.009  0.0094 0.0093 0.0094
 0.0082 0.007  0.0091 0.009  0.0093 0.0089 0.0096 0.009  0.0084 0.0076
 0.007  0.0053 0.0095 0.0098 0.0073 0.0085 0.0094 0.012  0.0031 0.0011
 0.002  0.0019 0.0042 0.0125 0.0059 0.0011 0.002  0.0004 0.0065 0.0115
 0.0138 0.0087 0.008

In [14]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 01:33:03    -2501.077938        0.435750
SciPyFminCG:    1 01:33:04    -2501.090813        0.120029


/tmp/ipykernel_10383/755014752.py:7: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 01:33:04    -2501.091027        0.120025
FIRE:    1 01:33:05    -2501.092958        0.090662
FIRE:    2 01:33:06    -2501.094246        0.086890
FIRE:    3 01:33:07    -2501.095104        0.065020
FIRE:    4 01:33:07    -2501.095963        0.068988
FIRE:    5 01:33:08    -2501.097250        0.083816
FIRE:    6 01:33:09    -2501.098323        0.085736
FIRE:    7 01:33:10    -2501.100254        0.074623
FIRE:    8 01:33:10    -2501.102829        0.064510
FIRE:    9 01:33:11    -2501.104975        0.095393
FIRE:   10 01:33:12    -2501.107121        0.065844
FIRE:   11 01:33:13    -2501.110125        0.096097
FIRE:   12 01:33:13    -2501.113558        0.170979
FIRE:   13 01:33:14    -2501.116776        0.228177
FIRE:   14 01:33:15    -2501.121926        0.276289
FIRE:   15 01:33:16    -2501.123857        0.207483
FIRE:   16 01:33:16    -2501.124287        0.195944
FIRE:   17 01:33:17    -2501.124930        0.162804
FIRE:   18 01:

In [ ]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write("relaxed_LLZO_Li_interface_15A_frozen.cif", final_structure_ase)
write("relaxed_LLZO_Li_interface_15A_frozen.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [17]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file("relaxed_LLZO_Li_interface_15A_frozen.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 24.48 Å
Lowest Li slab atom: 34.98 Å
LLZO top z: 34.85 Å
Li penetration into LLZO: -0.14 Å
